In [ ]:
from beamtest_analysis_helper import plot_resolution_table, plot_resolution_with_pulls
from collections import defaultdict
import pandas as pd
import numpy as np

## Figure configs

In [ ]:
board_to_analyze = [1,2,3]
fig_configs = {
    0: {
        'name': 'Barcelona 21',
        'short': 'BAR_21',
        'offset': 20,
        'HV': 200,
    },
    1: {
        'name': 'Barcelona 12',
        'short': 'BAR_12',
        'offset': 20,
        'HV': 120,
    },
    2: {
        'name': 'ET2.03 Pair 16',
        'short': 'PAIR_16',
        'offset': 20,
        'HV': 220,
    },
    3: {
        'name': 'Barcelona 20',
        'short': 'BAR_20',
        'offset': 20,
        'HV': 200,
    },
}

for id in board_to_analyze:
    fig_configs[id]['title'] = f"{fig_configs[id]['name']} HV{fig_configs[id]['HV']}V OS:{fig_configs[id]['offset']}"

In [ ]:
## final_df: time resolution dataframe
final_df = pd.read_csv('../resolution_CERNSept2024_Run_28_TOA250to500.csv')

## nevt_track_df: number of events dataframe
nevt_track_df = pd.read_csv('../CERNSept2024_Run_28_nevt_per_track.csv')

In [ ]:
final_df

In [ ]:
nevt_track_df

In [ ]:
merged_df = pd.merge(final_df, nevt_track_df, on=['row0', 'col0', 'row1', 'col1', 'row2', 'col2', 'row3', 'col3'])
merged_df.sort_values(by=['nevt'], ascending=False).reset_index(drop=True)

## Define condition by track pixel combinations from merged_df

In [ ]:
condition = (merged_df['row0'] == merged_df['row1']+1) & \
            (merged_df['col0'] == merged_df['col1']+1) & \
            (merged_df['row1'] == merged_df['row2']+1) & \
            (merged_df['col1'] == merged_df['col2']+1) & \
            (merged_df['row1'] == merged_df['row3']) & \
            (merged_df['col1'] == merged_df['col3'])

selected_data = merged_df[condition]

In [ ]:
# Define a function to check if tuple exists as a row
# def is_tuple_in_df(t, df):
#     return any(row.tolist() == list(t) for _, row in df.iterrows())

In [ ]:
# col_list, row_list = np.meshgrid(np.arange(16),np.arange(16))
# scan_list = list(zip(row_list.flatten(),col_list.flatten()))

# satisfy_df = merged_df[condition].sort_values(by=['nevt'], ascending=False).reset_index(drop=True)
# non_satisfy_df = merged_df[~condition].sort_values(by=['nevt'], ascending=False).reset_index(drop=True)

# # Filter elements not present
# missing_pixels_board0 = [item for item in scan_list if not is_tuple_in_df(item, satisfy_df[[f'row{board_to_analyze[0]}', f'col{board_to_analyze[0]}']])]
# missing_pixels_board1 = [item for item in scan_list if not is_tuple_in_df(item, satisfy_df[[f'row{board_to_analyze[1]}', f'col{board_to_analyze[1]}']])]
# missing_pixels_board2 = [item for item in scan_list if not is_tuple_in_df(item, satisfy_df[[f'row{board_to_analyze[2]}', f'col{board_to_analyze[2]}']])]

# missing_pix_dict = {
#     board_to_analyze[0]: defaultdict(list),
#     board_to_analyze[1]: defaultdict(list),
#     board_to_analyze[2]: defaultdict(list),
# }

# for board_num, missing_pixels in zip(board_to_analyze, [missing_pixels_board0, missing_pixels_board1, missing_pixels_board2]):
#     for missing_pixel in missing_pixels:
#         board_key = f'board{board_num}'
#         row_key = f'row{board_num}'
#         col_key = f'col{board_num}'
#         res_key = f'res{board_num}'
#         err_key = f'err{board_num}'

#         tmp_df = non_satisfy_df.loc[(non_satisfy_df[row_key] == missing_pixel[0]) & (non_satisfy_df[col_key] == missing_pixel[1])]
#         if not tmp_df.empty:
#             missing_pix_dict[board_num]['row'].append(tmp_df.iloc[0][row_key])
#             missing_pix_dict[board_num]['col'].append(tmp_df.iloc[0][col_key])
#             missing_pix_dict[board_num]['res'].append(tmp_df.iloc[0][res_key])
#             missing_pix_dict[board_num]['err'].append(tmp_df.iloc[0][err_key])

# id_want_drop = list(set([0,1,2,3]) - set(board_to_analyze))[0]
# selected_data = merged_df[condition]
# selected_data = selected_data.drop(columns=[f'row{id_want_drop}', f'col{id_want_drop}', 'nevt'])
# selected_data.reset_index(drop=True, inplace=True)

In [ ]:
# selected_data

In [ ]:
# from pathlib import Path

# current_dir = Path('./plot_scripts_for_approval')
# output_mother_dir = current_dir / 'etroc_TB_figs'
# output_mother_dir.mkdir(exist_ok=True)

# ### Now you need change the directory name per campaign
# ### Naming rule is this:
# ### <TB location>_TB_MonthYear
# ### E.g. desy_TB_Apr2024, cern_TB_Sep2023, fnal_TB_Jul2024

# output_campaign_dir = output_mother_dir / 'desy_TB_Apr2024'
# output_campaign_dir.mkdir(exist_ok=True)

In [ ]:
plot_resolution_table(input_df=selected_data, board_ids=board_to_analyze, tb_loc='cern_h8', fig_config=fig_configs,
                      min_resolution=10, max_resolution=100, slides_friendly = False, show_number=True)

In [ ]:
plot_resolution_with_pulls(input_df=selected_data, board_ids=board_to_analyze, board_names=chip_names, tb_loc='cern_h8',
                           fig_tag=chip_figtitles, hist_range=[10, 95], hist_bins=50, print_fit_results=False)

In [ ]:
plot_resolution_with_pulls(input_df=selected_data, board_ids=board_to_analyze, tb_loc='cern', fig_config=fig_configs,
                           hist_range=[10, 95], hist_bins=50, print_fit_results=False)